In [ ]:
import numpy as np
import xarray as xr
from affine import Affine

import rioxarray
from rioxarray import Convention

# Create sample data
data = np.random.rand(100, 100)
da = xr.DataArray(
    data,
    dims=["y", "x"],
    coords={
        "x": np.linspace(-180, 180, 100),
        "y": np.linspace(-90, 90, 100)
    }
)

transform = Affine(3.6, 0.0, -180.0, 0.0, -1.8, 90.0)
print("Sample data created")

## CF Convention

The CF convention stores geospatial metadata in grid_mapping coordinate variables.

In [ ]:
# Write CRS and transform using CF convention
da_cf = da.rio.write_crs("EPSG:4326", convention=Convention.CF)
da_cf = da_cf.rio.write_transform(transform, convention=Convention.CF)

print("CF Convention attributes:")
print(f"Grid mapping: {da_cf.attrs.get('grid_mapping')}")
print(f"Grid mapping coordinate: {list(da_cf.coords.keys())}")
print(f"Grid mapping attrs: {da_cf.coords['spatial_ref'].attrs.keys()}")
print(f"GeoTransform: {da_cf.coords['spatial_ref'].attrs.get('GeoTransform')}")

## Zarr Conventions

The Zarr conventions store geospatial metadata as direct attributes on the data array.

In [ ]:
# Write CRS and transform using Zarr conventions
da_zarr = da.rio.write_crs("EPSG:4326", convention=Convention.Zarr)
da_zarr = da_zarr.rio.write_transform(transform, convention=Convention.Zarr)

print("Zarr Convention attributes:")
print(f"proj:code: {da_zarr.attrs.get('proj:code')}")
print(f"spatial:transform: {da_zarr.attrs.get('spatial:transform')}")
print(f"zarr_conventions: {[c['name'] for c in da_zarr.attrs.get('zarr_conventions', [])]}")

## Zarr-Specific Methods

rioxarray provides specialized methods for working with Zarr conventions.

In [ ]:
# Write CRS in multiple Zarr formats
da_zarr_full = da.rio.write_zarr_crs("EPSG:4326", format="all")

print("Multiple CRS formats:")
print(f"proj:code: {da_zarr_full.attrs.get('proj:code')}")
print(f"proj:wkt2: {da_zarr_full.attrs.get('proj:wkt2')[:50]}...")
print(f"proj:projjson type: {type(da_zarr_full.attrs.get('proj:projjson'))}")

In [ ]:
# Write complete spatial metadata
da_spatial = da.rio.write_zarr_transform(transform)
da_spatial = da_spatial.rio.write_zarr_spatial_metadata()

print("Complete spatial metadata:")
print(f"spatial:dimensions: {da_spatial.attrs.get('spatial:dimensions')}")
print(f"spatial:shape: {da_spatial.attrs.get('spatial:shape')}")
print(f"spatial:bbox: {da_spatial.attrs.get('spatial:bbox')}")
print(f"spatial:registration: {da_spatial.attrs.get('spatial:registration')}")

In [ ]:
# Write everything at once
da_complete = da.rio.write_zarr_conventions(
    input_crs="EPSG:4326",
    transform=transform,
    crs_format="code"  # or "all" for multiple formats
)

print("Complete Zarr conventions:")
print(f"Has CRS: {'proj:code' in da_complete.attrs}")
print(f"Has transform: {'spatial:transform' in da_complete.attrs}")
print(f"Has dimensions: {'spatial:dimensions' in da_complete.attrs}")
print(f"Number of attributes: {len(da_complete.attrs)}")

## Global Convention Setting

You can set the default convention globally to avoid specifying it for each method call.

In [ ]:
# Set Zarr as the global default
with rioxarray.set_options(convention=Convention.Zarr):
    da_global = da.rio.write_crs("EPSG:4326")  # Uses Zarr convention
    da_global = da_global.rio.write_transform(transform)  # Uses Zarr convention
    
    print("Using global Zarr convention:")
    print(f"proj:code: {da_global.attrs.get('proj:code')}")
    print(f"spatial:transform: {da_global.attrs.get('spatial:transform')}")
    print(f"Has grid_mapping: {'grid_mapping' in da_global.attrs}")

## Reading with Different Conventions

The reading behavior follows the global convention setting.

In [ ]:
# Create data with both conventions
da_both = da.rio.write_crs("EPSG:4326", convention=Convention.CF)
da_both = da_both.rio.write_crs("EPSG:4326", convention=Convention.Zarr)

print("Data with both conventions:")
print(f"Has CF grid_mapping: {'grid_mapping' in da_both.attrs}")
print(f"Has Zarr proj:code: {'proj:code' in da_both.attrs}")

# Read using CF convention
with rioxarray.set_options(convention=Convention.CF):
    crs_cf = da_both.rio.crs
    print(f"\nReading with CF convention: {crs_cf}")

# Read using Zarr convention  
with rioxarray.set_options(convention=Convention.Zarr):
    crs_zarr = da_both.rio.crs
    print(f"Reading with Zarr convention: {crs_zarr}")

## Performance Comparison

Zarr conventions can be faster for reading metadata since they use direct attribute access instead of coordinate variable lookups.

In [ ]:
import time

# Create test data with both conventions
large_data = xr.DataArray(
    np.random.rand(1000, 1000),
    dims=["y", "x"],
    coords={"x": range(1000), "y": range(1000)}
)

# Add CF metadata
cf_data = large_data.rio.write_crs("EPSG:4326", convention=Convention.CF)

# Add Zarr metadata 
zarr_data = large_data.rio.write_crs("EPSG:4326", convention=Convention.Zarr)

# Time CF reading
with rioxarray.set_options(convention=Convention.CF):
    start = time.time()
    for _ in range(100):
        _ = cf_data.rio.crs
    cf_time = time.time() - start

# Time Zarr reading
with rioxarray.set_options(convention=Convention.Zarr):
    start = time.time()
    for _ in range(100):
        _ = zarr_data.rio.crs
    zarr_time = time.time() - start

print(f"CF convention reading time: {cf_time:.4f} seconds")
print(f"Zarr convention reading time: {zarr_time:.4f} seconds")
print(f"Speedup: {cf_time / zarr_time:.2f}x")